In [ ]:
%%capture
!pip install wandb
import wandb
wandb.login(key ="9c0cf96a5b886197f51883781f17b735b2ac32b1")

In [ ]:

# !wget 'https://drive.google.com/uc?export=download&id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw' -O dataset.zip && unzip dataset.zip

In [ ]:

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()


class Lang:
    def __init__(self, name):
        self.name = name
        self.char2index = {'#': 0, '$': 1, '^': 2}
        self.char2count = {'#': 1, '$': 1, '^': 1}
        self.index2char = {0: '#', 1: '$', 2: '^'}
        self.n_chars = 3  # Count
        self.data = {}
        

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1



class EncodedData:
  def __init__(self,lang):
    self.train_df = pd.read_csv(f"/kaggle/input/dlassgn/hin_train.csv", header = None)
    self.val_df = pd.read_csv(f"/kaggle/input/dlassgn/hin_valid.csv", header = None)
    self.test_df = pd.read_csv(f"/kaggle/input/dlassgn/hin_test.csv", header = None)

    self.input_lang = Lang('eng')
    self.output_lang = Lang(lang)

    # add the words to the respective languages
    for i in range(len(self.train_df)):
        
        self.input_lang.addWord(self.train_df[0][i])
        self.output_lang.addWord(self.train_df[1][i])
    
    max_len_1,max_len_2 = self.max_word_lengths(self.train_df)
    self.train_xE,self.train_yE = self.preprocessing(self.train_df,max_len_1,max_len_2)

    max_len_1,max_len_2 = self.max_word_lengths(self.val_df)
    self.val_xE,self.val_yE = self.preprocessing(self.val_df,max_len_1,max_len_2)

    max_len_1,max_len_2 = self.max_word_lengths(self.test_df)
    self.test_xE,self.test_yE = self.preprocessing(self.test_df,max_len_1,max_len_2)

    print(type(self.train_xE))
    
  def max_word_lengths(self,df):
    max_len_1 = df.iloc[:,0].str.len().max()
    max_len_2 = df.iloc[:,1].str.len().max()
    return max_len_1, max_len_2



  def preprocessing(self,df,max_len_1,max_len_2):
    
    M1 = np.zeros((len(df),max_len_1+1))
    M2 = np.zeros((len(df),max_len_2+2))

    #print(M2.shape)
    
    # Encode column 1
    col1 = df.iloc[:, 0].str.lower().str.split()

    for i in range(len(col1)) :
      word = col1[i][0]
      # print(word," " ,word[0],word[1])
      for j in range(len(word)):
        char =word[j]
        if(self.input_lang.char2index.get(char) is not None):
          M1[i][j]=self.input_lang.char2index.get(char)
        else:
          M1[i][j]=1
    # print(col1)
    #encoded_col1 = np.array([encode(words, dict1, max_len_1) for words in col1])
    
    # Encode column 2
    col2 = df.iloc[:, 1].str.lower().str.split()
    # print((col2[0]))
    for i in range(len(col2)) :
      word = col2[i][0]
      # print(word," " ,word[0],word[1])
      M2[i][0]=2
      for j in range(len(word)):
        char =word[j]
        if(self.output_lang.char2index.get(char) is not None):
          M2[i][j+1]=self.output_lang.char2index.get(char)
        else:
          M2[i][j+1]=1
      
    
    
    return torch.from_numpy(M1),torch.from_numpy(M2)


class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, num_encoders, hidden_size, bidirectional=False, cell='GRU',dropout=0.0):
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_encoders = num_encoders
        self.bidirectional = bidirectional
        self.cell = cell
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout)

        if self.cell == 'RNN':
            self.encoder_rnn = nn.RNN(embedding_size, hidden_size, num_layers=num_encoders,
                                      bidirectional=bidirectional )
        
        elif self.cell == 'GRU':
            self.encoder_rnn = nn.GRU(embedding_size, hidden_size, num_layers=num_encoders,
                                      bidirectional=bidirectional)
        else:
            self.encoder_rnn = nn.LSTM(embedding_size, hidden_size, num_layers=num_encoders,
                                       bidirectional=bidirectional )
    def init_hidden(self, batch_size):
        num_directions = 2 if self.bidirectional else 1
        if self.cell == 'LSTM':
            return (torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device),
                    torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device))
        else:
          return torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device)
        
    def forward(self, input_seq):
        batch_size = input_seq.size(1)
        
        # Embedding layer
        embedded = self.embedding(input_seq.long())
        
        embedded = self.dropout(embedded)
        # Encoder layers
        encoder_hidden = self.init_hidden(batch_size)

        # print(encoder_hidden.shape)
        if self.cell == 'LSTM':
            (encoder_hidden,encoder_cell) = self.init_hidden(batch_size)
            encoder_outputs, (encoder_hidden, encoder_cell) = self.encoder_rnn(embedded, (encoder_hidden, encoder_cell))
            encoder_hidden = (encoder_hidden,encoder_cell)
        else:
          
          encoder_outputs, encoder_hidden = self.encoder_rnn(embedded, encoder_hidden)
        
        
        return  encoder_outputs,encoder_hidden



class DecoderRNN(nn.Module):
    def __init__(self, output_size, embedding_size, num_encoders, num_decoders, hidden_size, dropout=0.0, cell='GRU'):
        super(DecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_encoders = num_encoders
        self.num_decoders = num_decoders
        
        self.cell = cell
        self.embedding_size = embedding_size
        self.dropout = nn.Dropout(dropout)
        self.decoder_fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.embedding = nn.Embedding(output_size, embedding_size)
        

        if self.cell == 'RNN':
            self.decoder_rnn = nn.RNN(embedding_size, hidden_size, num_layers=num_decoders)
        elif self.cell == 'GRU':
            self.decoder_rnn = nn.GRU(embedding_size, hidden_size, num_layers=num_decoders)
                                      
        else :
            self.decoder_rnn = nn.LSTM(embedding_size, hidden_size, num_layers=num_decoders)                      
                                     


    def forward(self, input_seq, hidden):
        batch_size = input_seq.shape[0]
        

        
        decoder_input = (input_seq)

        decoder_input = decoder_input.unsqueeze(1)
        
        # Embedding layer
        embedded = self.embedding(decoder_input.long()).view(-1,batch_size,self.embedding_size)
        embedded = self.dropout(embedded)
        decoder_hidden = hidden

        decoder_output , decoder_hidden = self.decoder_rnn(embedded, decoder_hidden)
       
        # Project the output to the output size and apply softmax

        decoder_output = self.softmax(self.decoder_fc(decoder_output))
        
        # Return the output tensor
        return decoder_output,decoder_hidden

def count_exact_matches(pred, target):
    """
    Counts the number of rows in preds tensor that match exactly with each row in y tensor.
    preds: tensor of shape (batch_size, seq_len)
    y: tensor of shape (batch_size, seq_len)
    """
    pred = pred[:, 1:-1] # ignore first and last index of each row
    target = target[:, 1:-1] # ignore first and last index of each row
    count=0;
    for i in range(pred.shape[0]):
      flag = True
      for j in range(target.shape[1]):
        if(target[i][j]!=0 and target[i][j]!=pred[i][j]):
          flag=False
          break;
         
      if(flag):
        count+=1;
    
    return count

def evaluate(X,Y,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type):
    encoder.eval()
    decoder.eval()


    running_loss =0
    correct=0
    total=0
    Data = TensorDataset(X,Y)
    Loader = DataLoader(Data,batch_size=batch_size,shuffle=False) 
    loss_fun=nn.CrossEntropyLoss(reduction="sum")
    with torch.no_grad():
      for j,(x,y) in enumerate(Loader):
        loss=0

        x = x.to(device)
        y = y.to(device)

        x = x.transpose(0,1)
        y = y.transpose(0,1)
        encoder_output, encoder_hidden = encoder(x)
        
        if bidirectional:
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor
          
        
        
        decoder_input =(y)[0]
        # print("encoder_hidden:-",encoder_hidden.shape)
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        pred_output=torch.zeros(y.shape[0],batch_size,output_size)
          

        for k in range(1,y.shape[0]):

          
          decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          decoder_input=indices.clone()

        decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
        pred_output[-1]=decoder_output
        # print(pred_output.shape)
        # print(train_y.shape)
        
        pred_output= pred_output.view(-1, pred_output.size(-1))  # Reshape to (batch_size * seq_len, vocab_size)
        y = y.reshape(-1)

        pred_output = pred_output.to(device)
        y =y.to(device)

        loss = loss_fun(pred_output, y.long())
        running_loss += loss.item()

        # Compute the gradients and update the parameters
        

        # Compute the accuracy
        preds = torch.argmax(pred_output, dim=1)

        preds = preds.reshape(batch_size,-1)
        y = y.reshape(batch_size,-1)
        
        # print("------------------Iter next--------------")
        # print(preds[:][0])
        # print(train_y[0])
        
        # print("-------------------------PRED----------------------")
        # print(preds)
        # print("-------------------------Train---------------------")
        # print(train_y)
        correct += count_exact_matches(preds,y)
          # Print the training progress
          
    avg_loss = running_loss / (batch_size * (X.shape[0]/batch_size))
  
    avg_acc = correct / X.shape[0]
    
    return avg_loss , avg_acc

            
   
 

def generateDecoderHidden(cell,num_encoder,num_decoder,encoder_hidden):
    hidden={}
    if(cell=="LSTM"):
      
      hx,state = (encoder_hidden)
      if(num_encoder>num_decoder):
        hx = hx[num_encoder-num_decoder:]
        state = state[num_encoder-num_decoder:]
      elif(num_encoder<num_decoder):
        top = hx[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        hiddenExtra =top.repeat(extra,1,1)
        hx =torch.cat((hx,hiddenExtra),dim=0)
      
        stop = state[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        stateExtra =top.repeat(extra,1,1)
        state =torch.cat((state,stateExtra),dim=0)

      hidden=(hx,state)
    else :

      hidden = encoder_hidden
          
      if(num_encoder>num_decoder):
        hidden = encoder_hidden[num_encoder-num_decoder:]
        
      elif(num_encoder<num_decoder):
        top = encoder_hidden[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        hiddenExtra =top.repeat(extra,1,1)
        hidden =torch.cat((encoder_hidden,hiddenExtra),dim=0)
    return hidden






In [ ]:
print(device)

In [ ]:
# get encoded data of perticular language

data = EncodedData("hin")    

from torch.utils.data import TensorDataset






In [ ]:
learn_rate = 0.001
batch_size = 16
hidden_size = 128
embedding_size = 128
num_encoder = 3
num_decoder = 3
cell_type = 'RNN'
bidirectional =True
dropout = 0.3
teach_ratio = 0.5
epochs = 10


In [ ]:


def trainRNNandGRU(data,cell_type="RNN",embedding_size=64,num_encoder=2,num_decoders=2,hidden_size=32,batch_size=16,bidirectional=False,dropout=0.0,beam_width=0,epoch=10,learn_rate=0.001,teacher_ratio=0.0):
  input_size = data.input_lang.n_chars
  output_size = data.output_lang.n_chars
  encoder = EncoderRNN( input_size, embedding_size, num_encoder, hidden_size, bidirectional, cell_type,dropout)
  decoder = DecoderRNN(output_size,embedding_size,num_encoder,num_decoder,hidden_size,dropout,cell_type)

  trainData = TensorDataset(data.train_xE, data.train_yE)
  trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True) 



  encoder_optimizer=optim.Adam(encoder.parameters(),learn_rate)
  decoder_optimizer=optim.Adam(decoder.parameters(),learn_rate)
  loss_fun=nn.CrossEntropyLoss(reduction="sum")

  encoder.to(device)
  decoder.to(device)

  for i in range(epochs):
  
    running_loss = 0.0
    train_correct = 0

    encoder.train()
    decoder.train()

    for j,(train_x,train_y) in enumerate(trainLoader):
        loss=0
        train_x = train_x.to(device)
        train_y = train_y.to(device)

        train_x = train_x.transpose(0, 1)
        train_y = train_y.transpose(0, 1)
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_output, encoder_hidden = encoder(train_x)
        
        # print(encoder_hidden.shape)
        # # lets move to the decoder
        # if(bidirectional):
        #     split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
        #     encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
        

        
        if bidirectional:
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor
        
        # Compute the loss and accuracy
        
        decoder_input =(train_y)[0]
        # print("encoder_hidden:-",encoder_hidden.shape)
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        pred_output=torch.zeros(train_y.shape[0],batch_size,output_size)
          
        
        for k in range(1,train_y.shape[0]):

          
          decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          # print(indices)
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          
          if((random.random()<teach_ratio)):
            decoder_input=(train_y)[k]
          else:
            decoder_input=indices.clone()

        decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
        pred_output[-1]=decoder_output
        

        pred_output= pred_output.view(-1, pred_output.size(-1))  # Reshape to (batch_size * seq_len, vocab_size)
        train_y = train_y.reshape(-1)

        pred_output = pred_output.to(device)
        train_y = train_y.to(device)
        loss = loss_fun(pred_output, train_y.long())
        
        running_loss += loss.item()

        # Compute the gradients and update the parameters
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        # Compute the accuracy
        # print(pred_output.shape)
        preds = torch.argmax(pred_output, dim=1)
        
        preds = preds.reshape(batch_size,-1)
        train_y = train_y.reshape(batch_size,-1)
        
        
        train_correct += count_exact_matches(preds,train_y)
       
        
    avg_loss = running_loss / (batch_size * (len(trainLoader)))
    avg_acc = train_correct/(batch_size*(len(trainLoader)))
    
#     print('Epoch [{}/{}], Step [{}/{}], Train_Loss: {:.4f}, Accuracy :{:.4f}'
#           .format(i+1, epochs, j+1, len(trainLoader), avg_loss,avg_acc*100))
    # evaluate(data.val_xE,data.val_yE,encoder,decoder,device,output_size,batch_size,cell_type)
    
            
    val_loss,val_acc = evaluate(data.val_xE,data.val_yE,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type)
#     print("val_loss:-",val_loss,"val_acc :-",val_acc)
    wandb.log({'Train_loss': avg_loss, 'Train_acc': avg_acc*100,'Epoch':i+1,'Val_loss':val_loss,'Val_Acc':val_acc})
  return encoder ,decoder

# learn_rate = 0.01
# batch_size = 64
# hidden_size = 16
# embedding_size = 64
# num_encoder = 4
# num_decoder = 2
# cell_type = 'RNN'
# bidirectional =True
# dropout = 0.2
# teach_ratio = 0.7
# epochs = 10
# beam_width =0
# encoder,decoder=trainRNNandGRU(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epochs,learn_rate,teach_ratio)

  


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [ ]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding_size , output_size, num_decoder, cell_type, dropout=0.0):
        super(AttentionDecoderRNN, self).__init__()

        # Define the input size, hidden size, output size, and number of layers
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_decoder
        self.cell_type = cell_type
        # Define the embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.fc = nn.Linear(hidden_size, output_size)
        # Define the attention mechanism
        self.attention = nn.Linear(hidden_size + embedding_size, hidden_size)
        self.context_vector = nn.Parameter(torch.zeros(hidden_size))
        self.energy = nn.Linear(hidden_size * 2, 1)
        self.softmax = nn.Softmax(dim=0)
        # Define the output layer
        self.output_layer = nn.Linear(hidden_size, output_size)
        
        self.relu = nn.ReLU()
        # Define the RNN layer
        if cell_type == "RNN":
            self.decoder_rnn = nn.RNN(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)
        elif cell_type == "GRU":
            self.decoder_rnn = nn.GRU(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)
        elif cell_type == "LSTM":
            self.decoder_rnn = nn.LSTM(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)
        else:
            raise ValueError("Invalid cell type")

        # Define the dropout layer
        self.dropout = nn.Dropout(dropout)

        # Define softmax
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs):
        # Compute the attention 
        input = input.unsqueeze(0)

        seq_len = encoder_outputs.shape[0]

        embedding = self.embedding(input.long())
        embedding = self.dropout(embedding)



        cell={}
        if(self.cell_type=="LSTM"):
          hidden ,cell= hidden 
       
        hidden_row = hidden[0].unsqueeze(0)  # add a new dimension
        hidden_new= hidden_row.repeat(seq_len-self.num_layers, 1,  1)  # repeat along the new dimension
        # hidden = hidden.view(seq_len, batch_size, -1)  # reshape to (seq_len, batch_size, hidden_size)
        hidden_cat = torch.cat([hidden, hidden_new], dim=0)
        
        energy = self.relu(self.energy(torch.cat((hidden_cat, encoder_outputs), dim=2)))
        # print(energy.shape)

        attention = self.softmax(energy)
        context_vector = torch.einsum("snk,snl->knl", attention, encoder_outputs)
        rnn_input = torch.cat((context_vector, embedding), dim=2)


        if(self.cell_type=="LSTM"):
          hidden = (hidden,cell)
        

        outputs, decoder_hidden= self.decoder_rnn(rnn_input, hidden)
        # print(outputs.shape)
        predictions = self.fc(outputs)
        # print(predictions.shape)
        output = self.logsoftmax(predictions)
        
        return output, decoder_hidden


In [ ]:

def trainAttension(data,cell_type="RNN",embedding_size=64,num_encoder=2,num_decoders=2,hidden_size=32,batch_size=16,bidirectional=False,dropout=0.0,beam_width=0,epoch=10,learn_rate=0.001,teacher_ratio=0.0):
  input_size = data.input_lang.n_chars
  output_size = data.output_lang.n_chars
  encoder = EncoderRNN( input_size, embedding_size, num_encoder, hidden_size, bidirectional, cell_type,dropout)
  decoder = AttentionDecoderRNN(cell_type=cell_type,dropout=dropout,embedding_size=embedding_size,hidden_size=hidden_size,num_decoder=num_decoder,output_size=output_size)

  trainData = TensorDataset(data.train_xE, data.train_yE)
  trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True) 



  encoder_optimizer=optim.Adam(encoder.parameters(),learn_rate)
  decoder_optimizer=optim.Adam(decoder.parameters(),learn_rate)
  loss_fun=nn.CrossEntropyLoss(reduction="sum")

  encoder.to(device)
  decoder.to(device)

  for i in range(epochs):
  
    running_loss = 0.0
    correct = 0

    encoder.train()
    decoder.train()

    for j,(train_x,train_y) in enumerate(trainLoader):
        loss=0
        train_x = train_x.to(device)
        train_y = train_y.to(device)

        train_x = train_x.transpose(0, 1)
        train_y = train_y.transpose(0, 1)
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_output, encoder_hidden = encoder(train_x)
        
        # print(encoder_output.shape)
        # # lets move to the decoder
        # if(bidirectional):
        #     split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
        #     encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
        

        
        if bidirectional:
          hidden_size = encoder_output.size(2) // 2
          forward_output = encoder_output[:, :, :hidden_size]
          backward_output = encoder_output[:, :, hidden_size:]
          encoder_output = torch.add(forward_output, backward_output)/2

          
        
          
          # print(hidden_size)
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor
          
        # Compute the loss and accuracy
        
        decoder_input =(train_y)[0]
        # print("encoder_hidden:-",encoder_hidden.shape)
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        pred_output=torch.zeros(train_y.shape[0],batch_size,output_size)
          
        
        for k in range(1,train_y.shape[0]):

          
          decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden,encoder_output)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          # print(indices)
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          
          if((random.random()<teach_ratio)):
            decoder_input=(train_y)[k]
          else:
            decoder_input=indices.clone()

        decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden,encoder_output)
          
        pred_output[-1]=decoder_output
        

        pred_output= pred_output.view(-1, pred_output.size(-1))  # Reshape to (batch_size * seq_len, vocab_size)
        train_y = train_y.reshape(-1)

        pred_output = pred_output.to(device)
        train_y = train_y.to(device)
        loss = loss_fun(pred_output, train_y.long())
        
        running_loss += loss.item()

        # Compute the gradients and update the parameters
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        # Compute the accuracy
        # print(pred_output.shape)
        preds = torch.argmax(pred_output, dim=1)

        
        preds = preds.reshape(batch_size,-1)
        train_y = train_y.reshape(batch_size,-1)
        
        
        correct += count_exact_matches(preds,train_y)
        # print("------------------Iter next--------------")
        # print(preds[:][0])
        # print(train_y[0])
        # print(preds.shape)
        # print(train_y.shape)
        # print("-------------------------PRED----------------------")
        # print(preds)
        # print("-------------------------Train---------------------")
        # print(train_y)
        
        # Print the training progress
        
    avg_loss = running_loss / (batch_size * (len(trainLoader)))
    avg_acc = correct/(batch_size*(len(trainLoader)))
#     print('Epoch [{}/{}], Step [{}/{}], Train_Loss: {:.4f}, Accuracy :{:.4f}'
#           .format(i+1, epochs, j+1, len(trainLoader), avg_loss,avg_acc*100))
#     # evaluate(data.val_xE,data.val_yE,encoder,decoder,device,output_size,batch_size,cell_type)
    running_loss = 0.0
    correct = 0
            
    val_loss,val_acc = evaluateAttension(data.val_xE,data.val_yE,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type)
    wandb.log({'Train_loss': avg_loss, 'Train_acc': avg_acc*100,'Epoch':i+1,'Val_loss':val_loss,'Val_Acc':val_acc})
  return encoder ,decoder


def evaluateAttension(X,Y,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type):
    encoder.eval()
    decoder.eval()


    running_loss =0
    correct=0
    total=0
    Data = TensorDataset(X,Y)
    Loader = DataLoader(Data,batch_size=batch_size,shuffle=False) 
    loss_fun=nn.CrossEntropyLoss(reduction="sum")
    with torch.no_grad():
      for j,(x,y) in enumerate(Loader):
        loss=0

        x = x.to(device)
        y = y.to(device)

        x = x.transpose(0,1)
        y = y.transpose(0,1)
        encoder_output, encoder_hidden = encoder(x)
        
        if bidirectional:
          hidden_size = encoder_output.size(2) // 2
          forward_output = encoder_output[:, :, :hidden_size]
          backward_output = encoder_output[:, :, hidden_size:]
          encoder_output = torch.add(forward_output, backward_output)/2

          # print(hidden_size)
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor
          
        
        
        decoder_input =(y)[0]
        # print("encoder_hidden:-",encoder_hidden.shape)
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        pred_output=torch.zeros(y.shape[0],batch_size,output_size)
          

        for k in range(1,y.shape[0]):

          
          decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden,encoder_output)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          decoder_input=indices.clone()

        decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden,encoder_output)
          
        pred_output[-1]=decoder_output
        # print(pred_output.shape)
        # print(train_y.shape)
        
        pred_output= pred_output.view(-1, pred_output.size(-1))  # Reshape to (batch_size * seq_len, vocab_size)
        y = y.reshape(-1)

        pred_output = pred_output.to(device)
        y =y.to(device)

        loss = loss_fun(pred_output, y.long())
        running_loss += loss.item()

        # Compute the gradients and update the parameters
        

        # Compute the accuracy
        preds = torch.argmax(pred_output, dim=1)

        preds = preds.reshape(batch_size,-1)
        y = y.reshape(batch_size,-1)
        
        # print("------------------Iter next--------------")
        # print(preds[:][0])
        # print(train_y[0])
        
        # print("-------------------------PRED----------------------")
        # print(preds)
        # print("-------------------------Train---------------------")
        # print(train_y)
        correct += count_exact_matches(preds,y)
          # Print the training progress
          
    avg_loss = running_loss / (batch_size * (X.shape[0]/batch_size))
  
    avg_acc = 100*correct / X.shape[0]
    
    return avg_loss , avg_acc
    
# encoder,decoder=trainAttension(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epochs,learn_rate,teach_ratio)

  


In [ ]:
def wandb_runs(data):
    
    config = {
        "project":"CS6910_Assignment3",
        "method": 'bayes',
        "metric": {
        'name': 'acc',
        'goal': 'maximize'
        },
        'parameters' :{
        "epochs": {"values":[5,10,20]},
        "learn_rate": {"values":[0.01,0.001,0.0001]},
        "batch_size": {"values": [64,128,256]},
        "embedding_size": {"values":[16, 32, 64, 256, 512]},
        "hidden_size": {"values":[16, 32, 64, 256, 512]},
        "encoder_layers": {"values":[2,3,4]},
        "decoder_layers": {"values":[2,3,4]},
        "cell_type": {"values":["RNN","LSTM","GRU"]},
        "bi_directional":{"values":["Yes","No"]},
        "dropout":{"values":[0.1,0.2,0.5]},
        "attention":{"values":["Yes","No"]},
        "teach_ratio":{"values":[0.2,0.5,0.7]}
        
        }
    }
    def train_rnn():
        wandb.init()
        
       

        name='_CT_'+str(wandb.config.cell_type)+"_BS_"+str(wandb.config.batch_size)+"_EPOCH_"+str(wandb.config.epochs)+"_ES_"+str(wandb.config.embedding_size)+"_HS_"+str(wandb.config.hidden_size)
        wandb.run.name = name
        

        
        
        learn_rate = wandb.config.learn_rate
        batch_size = wandb.config.batch_size
        hidden_size = wandb.config.hidden_size
        embedding_size = wandb.config.embedding_size
        num_encoder = wandb.config.encoder_layers
        num_decoder = wandb.config.decoder_layers
        cell_type = wandb.config.cell_type
        bidirectional = (wandb.config.bi_directional=="Yes")
        dropout = wandb.config.dropout
        teach_ratio = wandb.config.teach_ratio
        epochs = wandb.config.epochs
        beam_width =0
        if wandb.config.attention == "Yes" :
            
            encoder,decoder=trainAttension(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epochs,learn_rate,teach_ratio)
        else:
            
            encoder,decoder=trainRNNandGRU(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epochs,learn_rate,teach_ratio)

          
        
        wandb.run.save()
        wandb.run.finish()
    sweep_id=wandb.sweep(config,project="CS6910_Assignment3")
    
    
    wandb.agent(sweep_id,function=train_rnn,count=10)
    wandb.agent(sweep_id,function=train_rnn,count=10)
    wandb.agent(sweep_id,function=train_rnn,count=10)
    wandb.agent(sweep_id,function=train_rnn,count=10)
    wandb.agent(sweep_id,function=train_rnn,count=10)
    wandb.agent(sweep_id,function=train_rnn,count=10)
    wandb.agent(sweep_id,function=train_rnn,count=10)
    wandb.agent(sweep_id,function=train_rnn,count=10)
    wandb.agent(sweep_id,function=train_rnn,count=10)


In [ ]:
wandb_runs(data)

In [ ]:
attention= Yes
batch_size=64
bi_directional=False
cell_type='RNN'
num_decoders4
dropout= 0.5
embedding_size= 256
num_encoders= 3
wandb: 	epochs: 10
wandb: 	hidden_size: 256
wandb: 	learn_rate: 0.0001
wandb: 	teach_ratio: 0.2

In [ ]:
attention= "Yes"
learn_rate = 0.0001
batch_size = 64
hidden_size = 256
embedding_size = 256
num_encoder = 3
num_decoder = 4
cell_type = 'RNN'
bidirectional =False
dropout = 0.5
teach_ratio = 0.2
epochs = 10
beam_width =0

In [ ]:
# if attention == "Yes" :
            
#     encoder,decoder=trainAttension(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epochs,learn_rate,teach_ratio)
# else:

#     encoder,decoder=trainRNNandGRU(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epochs,learn_rate,teach_ratio)


In [ ]:
def print_prediction()